In [1]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score
import pandas as pd
import numpy as np
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import sys
sys.path.insert(0, '..')
from Common import preprocessing,evaluation
classes_map = {'DOC':1, 'ENTER':2, 'ORG':3, 'PRIV':4, 'RANG':5, 'HOST':6}

In [2]:
df = pd.read_csv('..//Data//labeled.abiturient.ru.txt', delimiter=';', engine='python',encoding='utf8')

In [45]:
df

,question,class
0,"Здравствуйте, мед.справку нужно нести только в...",DOC
1,сколько дней рассматривают заявление в приемно...,DOC
2,Какие абитуриенты в первую очередь проходят на...,RANG
3,я подавала документы в электронном виде но до ...,DOC
4,Какие документы нужно отправить по почте?,DOC
5,"Подскажите пожалуйста, я послала документы по ...",DOC
6,"Подскажите, обязательно ли заполнять анкету на...",DOC
7,"Здравствуйте, подскажите, пожалуйста, у вас ес...",DOC
8,"Здравствуйте. У меня на сайте в списках, подав...",RANG
9,Здравствуйте! При формировании списка рекоменд...,RANG


In [3]:
questions = np.array(df.question)
questions = preprocessing.preprocess_list(questions)

documents = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(questions)]
model = Doc2Vec(documents,iter=50,vector_size=300)

#[model.infer_vector(["принести","аттестат","справка","нужный"])]
embeddings = []
for q in range(len(questions)):
    embeddings.append(model[q])

/home/alex/miniconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:531: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")


In [4]:
vectorizer = TfidfVectorizer(ngram_range=(1,1))
X = vectorizer.fit_transform(questions)

In [201]:
#X = np.array(pd.concat([pd.DataFrame(X.todense()),pd.DataFrame(embeddings)],axis=1))

In [5]:
classes = np.array(df['class'])
y = list(map(lambda x: classes_map[x],classes))

In [6]:
log_reg = OneVsRestClassifier(LogisticRegression(random_state=0,C=10,solver='lbfgs',)).fit(X, y)

In [7]:
def predict (question,model):
    question = preprocessing.preprocess_list([question])[0]
    vect = vectorizer.transform([question])
    return model.predict_proba(vect)

In [8]:
df_test = pd.read_csv('..//Data//labeled.pstu.ru.txt', delimiter=';', engine='python',encoding='utf8')
test_questions = np.array(df_test.question)
test_questions = preprocessing.preprocess_list(test_questions)
X_test = vectorizer.transform(test_questions)

test_embeddings = []
for i in test_questions:
    test_embeddings.append(model.infer_vector(i.split()))

#X_test = np.array(pd.concat([pd.DataFrame(X_test.todense()),pd.DataFrame(test_embeddings)],axis=1))
    
test_classes = np.array(df_test['class'])
y_test = list(map(lambda x: classes_map[x],test_classes))

In [18]:
evaluation.get_CV_scores(log_reg,X,y,'precision_micro').mean()

0.7942949160340463

In [10]:
def cosine_classifier(sentence):
    DOC = []
    ENTER = []
    ORG = []
    PRIV = []
    RANG = []
    HOST = []

    for i in range(X.shape[0]):
        if y[i] == 1:
            DOC.append(cosine_similarity([X[i].toarray()[0]],[vectorizer.transform([sentence]).toarray()[0]])[0][0])
        elif y[i] == 2:
            ENTER.append(cosine_similarity([X[i].toarray()[0]],[vectorizer.transform([sentence]).toarray()[0]])[0][0])
        elif y[i] == 3:
            ORG.append(cosine_similarity([X[i].toarray()[0]],[vectorizer.transform([sentence]).toarray()[0]])[0][0])
        elif y[i] == 4:
            PRIV.append(cosine_similarity([X[i].toarray()[0]],[vectorizer.transform([sentence]).toarray()[0]])[0][0])
        elif y[i] == 5:
            RANG.append(cosine_similarity([X[i].toarray()[0]],[vectorizer.transform([sentence]).toarray()[0]])[0][0])
        elif y[i] == 6:
            HOST.append(cosine_similarity([X[i].toarray()[0]],[vectorizer.transform([sentence]).toarray()[0]])[0][0])
        
        res = [np.mean(DOC),np.mean(ENTER),np.mean(ORG),np.mean(PRIV),np.mean(RANG),np.mean(HOST)]
    return res.index(max(res))+1

In [13]:
y_pred = []
for x in questions:
    y_pred.append(cosine_classifier(x))

/home/alex/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/alex/miniconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [20]:
precision_score(y_pred,y,average='micro')

0.8203221809169765

In [40]:
cosine_classifier('подавать документ электронный вид сей пора ответ почему')

/home/alex/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/alex/miniconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0